In [6]:
%pip install langchain langchain_community 

Note: you may need to restart the kernel to use updated packages.


In [64]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain.llms.openai import OpenAIChat
from langchain_core.output_parsers.string import StrOutputParser
import pandas as pd
from IPython.display import display, Markdown
llm = Ollama(model="phi3:latest", num_ctx=4096, num_predict=1024, temperature=0.1)
#llm = OpenAIChat(model="gpt-4", temperature=0.1)

In [102]:
question = "How to establish the genealogy of the families in the graph?"

# Tree of thoughts

In [66]:
import json

def load_graph(file):
    with open(file) as f:
        data = json.load(f)

    text = "```mermaid\ngraph LR\n"
    for item in data["links"]:
        text += f'\t{item["source"]} -- {item["value"]} --> {item["target"]}\n'
    text += "```"
    return text
text = load_graph("miserables_1.json")

graph_length = len(text)

print(f"Graph length: {graph_length}")


Graph length: 427


```mermaid
graph LR;
    A[Process Inputs] --> B[Create Thoughts];
    B --> C[Evaluate Pathways];
    C --> D[Summarize Results];
    D --> E[Formulate Conclusions];
```

In [74]:
#display(Markdown(text))
display(text)

'```mermaid\ngraph LR\n\tNapoleon -- 1 --> Myriel\n\tMlle.Baptistine -- 8 --> Myriel\n\tMme.Magloire -- 10 --> Myriel\n\tMme.Magloire -- 6 --> Mlle.Baptistine\n\tCountessdeLo -- 1 --> Myriel\n\tGeborand -- 1 --> Myriel\n\tChamptercier -- 1 --> Myriel\n\tCravatte -- 1 --> Myriel\n\tCount -- 2 --> Myriel\n\tOldMan -- 1 --> Myriel\n\tValjean -- 1 --> Labarre\n\tValjean -- 3 --> Mme.Magloire\n\tValjean -- 3 --> Mlle.Baptistine\n\tValjean -- 5 --> Myriel\n```'

# Process inputs

In [73]:
prompt_process_inputs = PromptTemplate.from_template("""
## 1. Process inputs:
    Given the following data, expressed as Mermaid graph. 
    Please count the number of nodes and links in the graph.
### Graph
```mermaid
{graph}
```
### Question
{question}
                                                         
""")

chain  = prompt_process_inputs | llm | StrOutputParser()

In [72]:
processedData =  chain.invoke({
    "graph": text,
    "question": question
})

display(Markdown(processedData))

In the given Mermaid graph, there are a total of **9 nodes** and **10 links**. Here's the breakdown:

Nodes (vertices):
- Napoleon
- Mlle.Baptistine
- Mme.Magloire
- CountessdeLo
- Geborand
- Champtercier
- Cravatte
- Myriel
- Valjean
- Labarre

Links (edges):
1. Napoleon --> Myriel
2. Mlle.Baptistine --> Myriel
3. Mme.Magloire --> Myriel
4. Mme.Magloire --> Mlle.Baptistine
5. CountessdeLo --> Myriel
6. Geborand --> Myriel
7. Champtercier --> Myriel
8. Cravatte --> Myriel
9. Count --> Myriel
1 CVajan -- Myriel
10. Valjean --> Labarre
11. Valjean --> Mme.Magloire
12. Valjean --> Mlle.Baptistine
13. Valjean --> Myriel

So, there are 9 nodes and 10 links in the graph.

# List Toughts

In [75]:
prompt_create_toughts = ChatPromptTemplate.from_template("""
## 2. Create list of toughts:
    Given the processed data, please create a list of toughts that you have about the data.
    Only list the that are relevant to the data and user question: {question}
    Do not answer the question yet.
    Please format your output as a 'JSON' array of strings.
### Processed data
{data}
                                                         
### Question
{question}
                                                         
""")

chain_toughts  = prompt_create_toughts | llm | StrOutputParser()

In [79]:
listOfThoughts = chain_toughts.invoke({
    "data": processedData,
    "question": question
    })

In [81]:
display(Markdown(listOfThoughts))

```json
[
    "To establish the genealogy of the families, one must analyze the connections between individuals and identify familial relationships.",
    "The nodes represent individual family members or entities within the graph.",
    "Links (edges) indicate a relationship or connection that could suggest a familial tie.",
    "Valjean's multiple links to various nodes may imply his role as an ancestor or relative in this genealogy.",
    "Myriel appears at several points, suggesting he might be central to the family structure within the graph."
]
```

#### Loop over elements

In [96]:
array = json.loads(listOfThoughts.replace("```json", "").replace("```", ""))
results = []
for item in array:
    results.append(listOfThoughts.invoke(item))

To establish the genealogy of the families, one must analyze the connections between individuals and identify familial relationships.
The nodes represent individual family members or entities within the graph.
Links (edges) indicate a relationship or connection that could suggest a familial tie.
Valjean's multiple links to various nodes may imply his role as an ancestor or relative in this genealogy.
Myriel appears at several points, suggesting he might be central to the family structure within the graph.


## Evaluate Pathways

In [82]:
prompt_evaluate = ChatPromptTemplate.from_template("""
## 3. Evaluate pathways:
    Given the thoughts and the data, please evaluate the pathways to  {question}.
    DO NOT answer the question yet.
    DO ONLY format your output as 'markdown'.
                                                   
### Processed data
{data}
                                                   
### List of thoughts
{thoughts}
                                                         
### Question
{question}

""")

chain_evaluate  = prompt_evaluate | llm | StrOutputParser()
evaluation = chain_evaluate.invoke({
    "data": text,
    "thoughts": listOfThoughts,
    "question": question
    })

display(Markdown(evaluation))

```markdown

To evaluate pathways for establishing the genealogy of the families in the given graph, we can follow these steps:

1. **Identify Key Nodes**: Begin by identifying nodes that appear multiple times or have numerous connections to other nodes as they may represent central figures like Myriel and Valjean.

2. **Analyze Direct Connections**: Examine direct links (edges) between individuals, such as the ones connecting Napoleon, Mlle. Baptistine, Mme. Magloire, Countess de Lo, Geborand, Champtercier, Cravatte, and Myriel to determine immediate familial relationships.

3. **Trace Ancestral Links**: Look for patterns where individuals like Valjean are connected to multiple nodes (e.g., Mme. Magloire, Mlle. Baptistine), which could indicate ancestral or relative ties.

4. **Consider Relative Proximity and Frequency of Connections**: The number of connections a node has can suggest the prominence of an individual within the family structure. For instance, Myriel's numerous links may imply his central role in this genealogy.

5. **Map Out Family Tree Structure**: Based on the analysis above, begin to map out potential relationships and construct a tree-like diagram that represents the families' genealogy within the graph.

6. **Validate Relationships**: Cross-reference identified familial connections with historical or contextual information (if available) to validate the proposed genealogical pathways.

```

## Synthesize Pathway Evaluation

In [97]:
prompt_synthesize_evaluate = ChatPromptTemplate.from_template("""
## 4. Evaluate pathways:
    Synthesize the data gathered from the inputs, thoughts list, and evaluation pathways
    Create a comprehensive plan for answering the question: {question}.
    DO NOT answer the question yet.
    DO ONLY format your output as 'JSON' dictionary of key value pairs.
                                                              
                                                   
### Processed data
{data}
                                                   
### List of thoughts
{thoughts}

### Pathways evaluation
{evaluation}
                                                         
### Question
{question}

""")

chain_synthesize_evaluate  = prompt_synthesize_evaluate | llm | StrOutputParser()
synthesis = chain_synthesize_evaluate.invoke({
    "data": text,
    "thoughts": listOfThoughts,
    "evaluation": evaluation,
    "question": question
    })


In [98]:

display(Markdown(synthesis))

```json
{
    "key_thoughts": [
        "Identify central figures",
        "Analyze direct familial connections",
        "Trace ancestral links",
        "Consider relative prominence",
        "Map family tree structure",
        "Validate relationships"
    ],
    "evaluation_pathways": [
        {
            "step": "Identify Key Nodes",
            "description": "Central figures like Myriel and Valjean are identified due to multiple connections."
        },
        {
            "step": "Analyze Direct Connections",
            "description": "Examine direct links between individuals for immediate familial relationships."
        },
        {
            "step": "Trace Ancestral Links",
            "description": "Look for patterns where Valjean is connected to multiple nodes, indicating ancestral ties."
        },
        {
            "step": "Consider Relative Proximity and Frequency of Connections",
            "description": "Assess the number of connections a node has as an indicator of individual prominence within the family structure."
        },
        {
            "step": "Map Out Family Tree Structure",
            "description": "Construct a tree-like diagram representing the genealogy based on analyzed data."
        },
        {
            "step": "Validate Relationships",
            "description": "Cross-reference identified connections with historical or contextual information to confirm accuracy."
        }
    ]
}
```

## Formulate Conclusions

In [107]:
prompt_formulate = ChatPromptTemplate.from_template("""
## 5. Formulate answer:
    Given the synthesized data, please formulate a final answer to the question: {question}.
    DO NOT mention thoughts or evaluation pathways, only formulate the final answer to the user.
    DO create a comprehensive and engaging article using markdown syntax, with sections and subsections.
                                                   
### Synthesis                                   
{synthesis}
                                                         
### Question
{question}

""")

chain_formulate  = prompt_formulate | llm | StrOutputParser()
synthesis = chain_formulate.invoke({
    "synthesis": synthesis,
    "question": question
    })

display(Markdown(synthesis))

To establish the genealogy of families in a graph, follow these steps:

1. **Identify Key Figures**: Begin by pinpointing central individuals who have numerous connections with other nodes within the graph. These key figures often represent ancestors or influential family members whose relationships span across various branches.

2. **Analyze Direct Connections**: Next, analyze direct familial links between pairs of nodes to determine parent-child and sibling relationships. Look for patterns such as repeated interactions that suggest a strong likelihood of kinship.

3. **Trace Ancestral Links**: Finally, trace ancestral connections by identifying recurring nodes that are connected to multiple individuals in the graph. These nodes likely represent common ancestors whose lineage can be traced through various family branches within the graph.

By following these steps, you will be able to construct a comprehensive genealogy of families depicted in the graph.